In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from src.utilities import *

In [3]:
raw_activity = pd.read_csv(os.path.join(raw_path, 'activity.csv'))
raw_activity=raw_activity.rename(columns = {'count':'counts'})
raw_activity.head()

,hcp,specialty,region,brand,month,channel,counts
0,39972,Internal medicine / pneumology,region_61,brand_1,2020-05,video,1
1,64026,General practicioner,region_168,brand_2,2020-05,phone,1
2,27892,Internal medicine,region_26,brand_2,2020-06,phone,1
3,33500,General practicioner,region_26,brand_2,2020-06,phone,1
4,61239,General practicioner,region_26,brand_2,2020-06,phone,1


In [4]:
def add_month(month):
    '''Given a string with month format of YYYY-MM add one month'''

    if (int(month[5:])<12):
        if (int(month[5:]) + 1 >= 10):
            return month[:5] + str(int(month[5:]) + 1)
        return month[:5] + '0' + str(int(month[5:]) + 1)
    return str(int(month[:4]) + 1) + '-01'

In [111]:
#activities to different specialty per month
activity_features_specialty = raw_activity.groupby(['month', 'region', 'brand', 
                                          'specialty']).counts.sum().unstack().reset_index().fillna(0)
activity_features_specialty['act_total_1mo'] = activity_features_specialty.sum(axis = 1)
activity_features_specialty['act_GP_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['General practicioner'].sum())
activity_features_specialty['act_IM_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Internal medicine'].sum())
activity_features_specialty['act_IMP_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Internal medicine / pneumology'].sum())
activity_features_specialty['act_IMGP_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Internal medicine and general practicioner'].sum())
activity_features_specialty['act_PE_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Pediatrician'].sum())

In [112]:
activity_features_specialty

specialty,month,region,brand,General practicioner,Internal medicine,Internal medicine / pneumology,Internal medicine and general practicioner,Pediatrician,act_total_1mo,act_GP_1mo,act_IM_1mo,act_IMP_1mo,act_IMGP_1mo,act_PE_1mo
0,2020-05,region_168,brand_2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1,2020-05,region_61,brand_1,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2,2020-06,region_112,brand_2,1.0,0.0,1.0,0.0,0.0,2.0,22.0,13.0,8.0,0.0,0.0
3,2020-06,region_157,brand_2,0.0,0.0,1.0,0.0,0.0,1.0,22.0,13.0,8.0,0.0,0.0
4,2020-06,region_26,brand_2,7.0,2.0,0.0,0.0,0.0,9.0,22.0,13.0,8.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4432,2021-09,region_96,brand_1,0.0,0.0,8.0,0.0,0.0,8.0,712.0,544.0,1173.0,15.0,58.0
4433,2021-09,region_96,brand_2,0.0,0.0,7.0,0.0,0.0,7.0,712.0,544.0,1173.0,15.0,58.0
4434,2021-09,region_97,brand_1,3.0,6.0,4.0,0.0,0.0,13.0,712.0,544.0,1173.0,15.0,58.0
4435,2021-09,region_97,brand_2,3.0,6.0,3.0,0.0,0.0,12.0,712.0,544.0,1173.0,15.0,58.0


In [98]:
#activities via different channel per month
activity_features_channel = raw_activity.groupby(['month', 'region', 'brand', 
                                          'channel']).counts.sum().unstack().reset_index().fillna(0)
activity_features_channel['act_f2f_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['f2f'].sum())
activity_features_channel['act_other_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['other'].sum())
activity_features_channel['act_phone_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['phone'].sum())
activity_features_channel['act_video_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['video'].sum())
activity_features_channel

channel,month,region,brand,f2f,other,phone,video,act_f2f_1mo,act_other_1mo,act_phone_1mo,act_video_1mo
0,2020-05,region_168,brand_2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,2020-05,region_61,brand_1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,2020-06,region_112,brand_2,0.0,0.0,2.0,0.0,0.0,0.0,41.0,2.0
3,2020-06,region_157,brand_2,0.0,0.0,1.0,0.0,0.0,0.0,41.0,2.0
4,2020-06,region_26,brand_2,0.0,0.0,9.0,0.0,0.0,0.0,41.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
4432,2021-09,region_96,brand_1,1.0,6.0,1.0,0.0,2226.0,145.0,80.0,51.0
4433,2021-09,region_96,brand_2,1.0,5.0,1.0,0.0,2226.0,145.0,80.0,51.0
4434,2021-09,region_97,brand_1,13.0,0.0,0.0,0.0,2226.0,145.0,80.0,51.0
4435,2021-09,region_97,brand_2,12.0,0.0,0.0,0.0,2226.0,145.0,80.0,51.0


In [113]:
#activity to different specialty per month per region
GP_monthly = activity_features_specialty.groupby(['month','region'])['General practicioner'].sum().reset_index()
GP_monthly.columns = ['month', 'region', 'act_GP_region_1mo']
IM_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine'].sum().reset_index()
IM_monthly.columns = ['month', 'region', 'act_IM_region_1mo']
IMP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine / pneumology'].sum().reset_index()
IMP_monthly.columns = ['month', 'region', 'act_IMP_region_1mo']
IMGP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine and general practicioner'].sum().reset_index()
IMGP_monthly.columns = ['month', 'region', 'act_IMGP_region_1mo']
PE_monthly = activity_features_specialty.groupby(['month','region'])['Pediatrician'].sum().reset_index()
PE_monthly.columns = ['month', 'region', 'act_PE_region_1mo']

activity_features_specialty = activity_features_specialty.merge(GP_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(IM_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(IMP_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(IMGP_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(PE_monthly, how = 'outer', on = ['month', 'region'])

activity_features_specialty

,month,region,brand,General practicioner,Internal medicine,Internal medicine / pneumology,Internal medicine and general practicioner,Pediatrician,act_total_1mo,act_GP_1mo,act_IM_1mo,act_IMP_1mo,act_IMGP_1mo,act_PE_1mo,act_GP_region_1mo,act_IM_region_1mo,act_IMP_region_1mo,act_IMGP_region_1mo,act_PE_region_1mo
0,2020-05,region_168,brand_2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2020-05,region_61,brand_1,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2020-06,region_112,brand_2,1.0,0.0,1.0,0.0,0.0,2.0,22.0,13.0,8.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,2020-06,region_157,brand_2,0.0,0.0,1.0,0.0,0.0,1.0,22.0,13.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2020-06,region_26,brand_2,7.0,2.0,0.0,0.0,0.0,9.0,22.0,13.0,8.0,0.0,0.0,7.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4432,2021-09,region_96,brand_1,0.0,0.0,8.0,0.0,0.0,8.0,712.0,544.0,1173.0,15.0,58.0,0.0,0.0,15.0,0.0,0.0
4433,2021-09,region_96,brand_2,0.0,0.0,7.0,0.0,0.0,7.0,712.0,544.0,1173.0,15.0,58.0,0.0,0.0,15.0,0.0,0.0
4434,2021-09,region_97,brand_1,3.0,6.0,4.0,0.0,0.0,13.0,712.0,544.0,1173.0,15.0,58.0,6.0,12.0,7.0,0.0,0.0
4435,2021-09,region_97,brand_2,3.0,6.0,3.0,0.0,0.0,12.0,712.0,544.0,1173.0,15.0,58.0,6.0,12.0,7.0,0.0,0.0


In [103]:
#activity via different channel per month per region
f2f_monthly = activity_features_channel.groupby(['month','region'])['f2f'].sum().reset_index()
f2f_monthly.columns = ['month', 'region', 'act_f2f_region_1mo']
phone_monthly = activity_features_channel.groupby(['month','region'])['phone'].sum().reset_index()
phone_monthly.columns = ['month', 'region', 'act_phone_region_1mo']
video_monthly = activity_features_channel.groupby(['month','region'])['video'].sum().reset_index()
video_monthly.columns = ['month', 'region', 'act_video_region_1mo']
other_monthly = activity_features_channel.groupby(['month','region'])['other'].sum().reset_index()
other_monthly.columns = ['month', 'region', 'act_other_region_1mo']

activity_features_channel = activity_features_channel.merge(f2f_monthly, how = 'outer', on = ['month', 'region'])
activity_features_channel = activity_features_channel.merge(phone_monthly, how = 'outer', on = ['month', 'region'])
activity_features_channel = activity_features_channel.merge(video_monthly, how = 'outer', on = ['month', 'region'])
activity_features_channel = activity_features_channel.merge(other_monthly, how = 'outer', on = ['month', 'region'])

activity_features_channel

In [30]:
# groupby = master.value_counts(['month', 'brand']).reset_index(name = 'monthly brand activity')
# master['monthly brand activity'] = groupby['monthly brand activity']

In [114]:
#activity to different specialty per month per brand
GP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['General practicioner'].sum().reset_index()
GP_monthly_brand.columns = ['month', 'brand', 'act_GP_brand_1mo']
IM_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine'].sum().reset_index()
IM_monthly_brand.columns = ['month', 'brand', 'act_IM_brand_1mo']
IMP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine / pneumology'].sum().reset_index()
IMP_monthly_brand.columns = ['month', 'brand', 'act_IMP_brand_1mo']
IMGP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine and general practicioner'].sum().reset_index()
IMGP_monthly_brand.columns = ['month', 'brand', 'act_IMGP_brand_1mo']
PE_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Pediatrician'].sum().reset_index()
PE_monthly_brand.columns = ['month', 'brand', 'act_PE_brand_1mo']

activity_features_specialty = activity_features_specialty.merge(GP_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(IM_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(IMP_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(IMGP_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(PE_monthly_brand, how = 'outer', on = ['month', 'brand'])

activity_features_specialty

,month,region,brand,General practicioner,Internal medicine,Internal medicine / pneumology,Internal medicine and general practicioner,Pediatrician,act_total_1mo,act_GP_1mo,act_IM_1mo,act_IMP_1mo,act_IMGP_1mo,act_PE_1mo,act_GP_region_1mo,act_IM_region_1mo,act_IMP_region_1mo,act_IMGP_region_1mo,act_PE_region_1mo,act_GP_brand_1mo,act_IM_brand_1mo,act_IMP_brand_1mo,act_IMGP_brand_1mo,act_PE_brand_1mo
0,2020-05,region_168,brand_2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2020-05,region_61,brand_1,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2020-06,region_112,brand_2,1.0,0.0,1.0,0.0,0.0,2.0,22.0,13.0,8.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,22.0,13.0,8.0,0.0,0.0
3,2020-06,region_157,brand_2,0.0,0.0,1.0,0.0,0.0,1.0,22.0,13.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22.0,13.0,8.0,0.0,0.0
4,2020-06,region_26,brand_2,7.0,2.0,0.0,0.0,0.0,9.0,22.0,13.0,8.0,0.0,0.0,7.0,2.0,0.0,0.0,0.0,22.0,13.0,8.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4432,2021-09,region_91,brand_2,32.0,15.0,3.0,0.0,3.0,53.0,712.0,544.0,1173.0,15.0,58.0,64.0,30.0,6.0,0.0,3.0,289.0,219.0,424.0,5.0,48.0
4433,2021-09,region_93,brand_2,0.0,2.0,0.0,0.0,0.0,2.0,712.0,544.0,1173.0,15.0,58.0,0.0,4.0,1.0,0.0,0.0,289.0,219.0,424.0,5.0,48.0
4434,2021-09,region_94,brand_2,0.0,2.0,3.0,0.0,0.0,5.0,712.0,544.0,1173.0,15.0,58.0,0.0,4.0,13.0,0.0,0.0,289.0,219.0,424.0,5.0,48.0
4435,2021-09,region_96,brand_2,0.0,0.0,7.0,0.0,0.0,7.0,712.0,544.0,1173.0,15.0,58.0,0.0,0.0,15.0,0.0,0.0,289.0,219.0,424.0,5.0,48.0


In [115]:
#activity via different channel per month per brand
f2f_monthly_brand = activity_features_channel.groupby(['month','brand'])['f2f'].sum().reset_index()
f2f_monthly_brand.columns = ['month', 'brand', 'act_f2f_brand_1mo']
phone_monthly_brand = activity_features_channel.groupby(['month','brand'])['phone'].sum().reset_index()
phone_monthly_brand.columns = ['month', 'brand', 'act_phone_brand_1mo']
video_monthly_brand = activity_features_channel.groupby(['month','brand'])['video'].sum().reset_index()
video_monthly_brand.columns = ['month', 'brand', 'act_video_brand_1mo']
other_monthly_brand = activity_features_channel.groupby(['month','brand'])['other'].sum().reset_index()
other_monthly_brand.columns = ['month', 'brand', 'act_other_brand_1mo']

activity_features_channel = activity_features_channel.merge(f2f_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_channel = activity_features_channel.merge(phone_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_channel = activity_features_channel.merge(video_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_channel = activity_features_channel.merge(other_monthly_brand, how = 'outer', on = ['month', 'brand'])

activity_features_channel

,month,region,brand,f2f,other,phone,video,act_f2f_1mo,act_other_1mo,act_phone_1mo,act_video_1mo,act_f2f_region_1mo,act_phone_region_1mo,act_video_region_1mo,act_other_region_1mo,act_f2f_brand_1mo,act_phone_brand_1mo,act_video_brand_1mo,act_other_brand_1mo
0,2020-05,region_168,brand_2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2020-05,region_61,brand_1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2020-06,region_112,brand_2,0.0,0.0,2.0,0.0,0.0,0.0,41.0,2.0,0.0,2.0,0.0,0.0,0.0,41.0,2.0,0.0
3,2020-06,region_157,brand_2,0.0,0.0,1.0,0.0,0.0,0.0,41.0,2.0,0.0,1.0,0.0,0.0,0.0,41.0,2.0,0.0
4,2020-06,region_26,brand_2,0.0,0.0,9.0,0.0,0.0,0.0,41.0,2.0,0.0,9.0,0.0,0.0,0.0,41.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4432,2021-09,region_91,brand_2,51.0,0.0,0.0,2.0,2226.0,145.0,80.0,51.0,99.0,0.0,4.0,0.0,881.0,22.0,23.0,59.0
4433,2021-09,region_93,brand_2,2.0,0.0,0.0,0.0,2226.0,145.0,80.0,51.0,5.0,0.0,0.0,0.0,881.0,22.0,23.0,59.0
4434,2021-09,region_94,brand_2,5.0,0.0,0.0,0.0,2226.0,145.0,80.0,51.0,17.0,0.0,0.0,0.0,881.0,22.0,23.0,59.0
4435,2021-09,region_96,brand_2,1.0,5.0,1.0,0.0,2226.0,145.0,80.0,51.0,2.0,2.0,0.0,11.0,881.0,22.0,23.0,59.0


In [116]:
activity_features = activity_features_channel.merge(activity_features_specialty, 
                                                    how = 'outer', on = ['month', 'region','brand'])



In [117]:
activity_features

,month,region,brand,f2f,other,phone,video,act_f2f_1mo,act_other_1mo,act_phone_1mo,act_video_1mo,act_f2f_region_1mo,act_phone_region_1mo,act_video_region_1mo,act_other_region_1mo,act_f2f_brand_1mo,act_phone_brand_1mo,act_video_brand_1mo,act_other_brand_1mo,General practicioner,Internal medicine,Internal medicine / pneumology,Internal medicine and general practicioner,Pediatrician,act_total_1mo,act_GP_1mo,act_IM_1mo,act_IMP_1mo,act_IMGP_1mo,act_PE_1mo,act_GP_region_1mo,act_IM_region_1mo,act_IMP_region_1mo,act_IMGP_region_1mo,act_PE_region_1mo,act_GP_brand_1mo,act_IM_brand_1mo,act_IMP_brand_1mo,act_IMGP_brand_1mo,act_PE_brand_1mo
0,2020-05,region_168,brand_2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2020-05,region_61,brand_1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2020-06,region_112,brand_2,0.0,0.0,2.0,0.0,0.0,0.0,41.0,2.0,0.0,2.0,0.0,0.0,0.0,41.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,22.0,13.0,8.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,22.0,13.0,8.0,0.0,0.0
3,2020-06,region_157,brand_2,0.0,0.0,1.0,0.0,0.0,0.0,41.0,2.0,0.0,1.0,0.0,0.0,0.0,41.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,22.0,13.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22.0,13.0,8.0,0.0,0.0
4,2020-06,region_26,brand_2,0.0,0.0,9.0,0.0,0.0,0.0,41.0,2.0,0.0,9.0,0.0,0.0,0.0,41.0,2.0,0.0,7.0,2.0,0.0,0.0,0.0,9.0,22.0,13.0,8.0,0.0,0.0,7.0,2.0,0.0,0.0,0.0,22.0,13.0,8.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4432,2021-09,region_91,brand_2,51.0,0.0,0.0,2.0,2226.0,145.0,80.0,51.0,99.0,0.0,4.0,0.0,881.0,22.0,23.0,59.0,32.0,15.0,3.0,0.0,3.0,53.0,712.0,544.0,1173.0,15.0,58.0,64.0,30.0,6.0,0.0,3.0,289.0,219.0,424.0,5.0,48.0
4433,2021-09,region_93,brand_2,2.0,0.0,0.0,0.0,2226.0,145.0,80.0,51.0,5.0,0.0,0.0,0.0,881.0,22.0,23.0,59.0,0.0,2.0,0.0,0.0,0.0,2.0,712.0,544.0,1173.0,15.0,58.0,0.0,4.0,1.0,0.0,0.0,289.0,219.0,424.0,5.0,48.0
4434,2021-09,region_94,brand_2,5.0,0.0,0.0,0.0,2226.0,145.0,80.0,51.0,17.0,0.0,0.0,0.0,881.0,22.0,23.0,59.0,0.0,2.0,3.0,0.0,0.0,5.0,712.0,544.0,1173.0,15.0,58.0,0.0,4.0,13.0,0.0,0.0,289.0,219.0,424.0,5.0,48.0
4435,2021-09,region_96,brand_2,1.0,5.0,1.0,0.0,2226.0,145.0,80.0,51.0,2.0,2.0,0.0,11.0,881.0,22.0,23.0,59.0,0.0,0.0,7.0,0.0,0.0,7.0,712.0,544.0,1173.0,15.0,58.0,0.0,0.0,15.0,0.0,0.0,289.0,219.0,424.0,5.0,48.0


In [120]:
activity_features=activity_features.rename(columns = {'f2f':'act_f2f_region_brand_1mo', 'other':'act_other_region_brand_1mo',
                                                     'phone':'act_phone_region_brand_1mo','video':'act_video_region_brand_1mo',
                                                     'General practicioner':'act_GP_region_brand_1mo', 'Internal medicine':'act_IM_region_brand_1mo',
                                                     'Internal medicine / pneumology':'act_IMP_region_brand_1mo',
                                                     'Internal medicine and general practicioner':'act_IMPG_region_brand_1mo',
                                                     'Pediatrician':'act_PE_region_brand_1mo'})

In [121]:
activity_features

,month,region,brand,act_f2f_region_brand_1mo,act_other_region_brand_1mo,act_phone_region_brand_1mo,act_video_region_brand_1mo,act_f2f_1mo,act_other_1mo,act_phone_1mo,act_video_1mo,act_f2f_region_1mo,act_phone_region_1mo,act_video_region_1mo,act_other_region_1mo,act_f2f_brand_1mo,act_phone_brand_1mo,act_video_brand_1mo,act_other_brand_1mo,act_GP_region_brand_1mo,act_IM_region_brand_1mo,act_IMP_region_brand_1mo,act_IMPG_region_brand_1mo,act_PE_region_brand_1mo,act_total_1mo,act_GP_1mo,act_IM_1mo,act_IMP_1mo,act_IMGP_1mo,act_PE_1mo,act_GP_region_1mo,act_IM_region_1mo,act_IMP_region_1mo,act_IMGP_region_1mo,act_PE_region_1mo,act_GP_brand_1mo,act_IM_brand_1mo,act_IMP_brand_1mo,act_IMGP_brand_1mo,act_PE_brand_1mo
0,2020-05,region_168,brand_2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2020-05,region_61,brand_1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2020-06,region_112,brand_2,0.0,0.0,2.0,0.0,0.0,0.0,41.0,2.0,0.0,2.0,0.0,0.0,0.0,41.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,22.0,13.0,8.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,22.0,13.0,8.0,0.0,0.0
3,2020-06,region_157,brand_2,0.0,0.0,1.0,0.0,0.0,0.0,41.0,2.0,0.0,1.0,0.0,0.0,0.0,41.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,22.0,13.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22.0,13.0,8.0,0.0,0.0
4,2020-06,region_26,brand_2,0.0,0.0,9.0,0.0,0.0,0.0,41.0,2.0,0.0,9.0,0.0,0.0,0.0,41.0,2.0,0.0,7.0,2.0,0.0,0.0,0.0,9.0,22.0,13.0,8.0,0.0,0.0,7.0,2.0,0.0,0.0,0.0,22.0,13.0,8.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4432,2021-09,region_91,brand_2,51.0,0.0,0.0,2.0,2226.0,145.0,80.0,51.0,99.0,0.0,4.0,0.0,881.0,22.0,23.0,59.0,32.0,15.0,3.0,0.0,3.0,53.0,712.0,544.0,1173.0,15.0,58.0,64.0,30.0,6.0,0.0,3.0,289.0,219.0,424.0,5.0,48.0
4433,2021-09,region_93,brand_2,2.0,0.0,0.0,0.0,2226.0,145.0,80.0,51.0,5.0,0.0,0.0,0.0,881.0,22.0,23.0,59.0,0.0,2.0,0.0,0.0,0.0,2.0,712.0,544.0,1173.0,15.0,58.0,0.0,4.0,1.0,0.0,0.0,289.0,219.0,424.0,5.0,48.0
4434,2021-09,region_94,brand_2,5.0,0.0,0.0,0.0,2226.0,145.0,80.0,51.0,17.0,0.0,0.0,0.0,881.0,22.0,23.0,59.0,0.0,2.0,3.0,0.0,0.0,5.0,712.0,544.0,1173.0,15.0,58.0,0.0,4.0,13.0,0.0,0.0,289.0,219.0,424.0,5.0,48.0
4435,2021-09,region_96,brand_2,1.0,5.0,1.0,0.0,2226.0,145.0,80.0,51.0,2.0,2.0,0.0,11.0,881.0,22.0,23.0,59.0,0.0,0.0,7.0,0.0,0.0,7.0,712.0,544.0,1173.0,15.0,58.0,0.0,0.0,15.0,0.0,0.0,289.0,219.0,424.0,5.0,48.0


In [13]:
def extract_activity_features(raw_activity: pd.DataFrame) -> pd.DataFrame:

    raw_activity.sort_values('month', inplace = True)
    
    #activities to different specialty per month
    activity_features_specialty = raw_activity.groupby(['month', 'region', 'brand', 
                                              'specialty']).counts.sum().unstack().reset_index().fillna(0)
    activity_features_specialty['act_total_1mo'] = activity_features_specialty.sum(axis = 1)
    activity_features_specialty['act_GP_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['General practicioner'].sum())
    activity_features_specialty['act_IM_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Internal medicine'].sum())
    activity_features_specialty['act_IMP_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Internal medicine / pneumology'].sum())
    activity_features_specialty['act_IMGP_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Internal medicine and general practicioner'].sum())
    activity_features_specialty['act_PE_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Pediatrician'].sum())
    
    #activities via different channel per month
    activity_features_channel = raw_activity.groupby(['month', 'region', 'brand', 
                                              'channel']).counts.sum().unstack().reset_index().fillna(0)
    activity_features_channel['act_f2f_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['f2f'].sum())
    activity_features_channel['act_other_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['other'].sum())
    activity_features_channel['act_phone_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['phone'].sum())
    activity_features_channel['act_video_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['video'].sum())
    activity_features_channel

    #activity to different specialty per month per region
    GP_monthly = activity_features_specialty.groupby(['month','region'])['General practicioner'].sum().reset_index()
    GP_monthly.columns = ['month', 'region', 'act_GP_region_1mo']
    IM_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine'].sum().reset_index()
    IM_monthly.columns = ['month', 'region', 'act_IM_region_1mo']
    IMP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine / pneumology'].sum().reset_index()
    IMP_monthly.columns = ['month', 'region', 'act_IMP_region_1mo']
    IMGP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine and general practicioner'].sum().reset_index()
    IMGP_monthly.columns = ['month', 'region', 'act_IMGP_region_1mo']
    PE_monthly = activity_features_specialty.groupby(['month','region'])['Pediatrician'].sum().reset_index()
    PE_monthly.columns = ['month', 'region', 'act_PE_region_1mo']

    activity_features_specialty = activity_features_specialty.merge(GP_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(IM_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(IMP_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(IMGP_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(PE_monthly, how = 'outer', on = ['month', 'region'])

    #activity to different specialty per month per brand
    GP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['General practicioner'].sum().reset_index()
    GP_monthly_brand.columns = ['month', 'brand', 'act_GP_brand_1mo']
    IM_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine'].sum().reset_index()
    IM_monthly_brand.columns = ['month', 'brand', 'act_IM_brand_1mo']
    IMP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine / pneumology'].sum().reset_index()
    IMP_monthly_brand.columns = ['month', 'brand', 'act_IMP_brand_1mo']
    IMGP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine and general practicioner'].sum().reset_index()
    IMGP_monthly_brand.columns = ['month', 'brand', 'act_IMGP_brand_1mo']
    PE_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Pediatrician'].sum().reset_index()
    PE_monthly_brand.columns = ['month', 'brand', 'act_PE_brand_1mo']

    activity_features_specialty = activity_features_specialty.merge(GP_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(IM_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(IMP_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(IMGP_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(PE_monthly_brand, how = 'outer', on = ['month', 'brand'])

    #activity via different channel per month per region
    f2f_monthly = activity_features_channel.groupby(['month','region'])['f2f'].sum().reset_index()
    f2f_monthly.columns = ['month', 'region', 'act_f2f_region_1mo']
    phone_monthly = activity_features_channel.groupby(['month','region'])['phone'].sum().reset_index()
    phone_monthly.columns = ['month', 'region', 'act_phone_region_1mo']
    video_monthly = activity_features_channel.groupby(['month','region'])['video'].sum().reset_index()
    video_monthly.columns = ['month', 'region', 'act_video_region_1mo']
    other_monthly = activity_features_channel.groupby(['month','region'])['other'].sum().reset_index()
    other_monthly.columns = ['month', 'region', 'act_other_region_1mo']

    activity_features_channel = activity_features_channel.merge(f2f_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_channel = activity_features_channel.merge(phone_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_channel = activity_features_channel.merge(video_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_channel = activity_features_channel.merge(other_monthly, how = 'outer', on = ['month', 'region'])



    #activity via different channel per month per brand
    f2f_monthly_brand = activity_features_channel.groupby(['month','brand'])['f2f'].sum().reset_index()
    f2f_monthly_brand.columns = ['month', 'brand', 'act_f2f_brand_1mo']
    phone_monthly_brand = activity_features_channel.groupby(['month','brand'])['phone'].sum().reset_index()
    phone_monthly_brand.columns = ['month', 'brand', 'act_phone_brand_1mo']
    video_monthly_brand = activity_features_channel.groupby(['month','brand'])['video'].sum().reset_index()
    video_monthly_brand.columns = ['month', 'brand', 'act_video_brand_1mo']
    other_monthly_brand = activity_features_channel.groupby(['month','brand'])['other'].sum().reset_index()
    other_monthly_brand.columns = ['month', 'brand', 'act_other_brand_1mo']

    activity_features_channel = activity_features_channel.merge(f2f_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_channel = activity_features_channel.merge(phone_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_channel = activity_features_channel.merge(video_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_channel = activity_features_channel.merge(other_monthly_brand, how = 'outer', on = ['month', 'brand'])

    
    activity_features = activity_features_channel.merge(activity_features_specialty, 
                                                    how = 'outer', on = ['month', 'region','brand'])


    activity_features=activity_features.rename(columns = {'f2f':'act_f2f_region_brand_1mo', 'other':'act_other_region_brand_1mo',
                                                     'phone':'act_phone_region_brand_1mo','video':'act_video_region_brand_1mo',
                                                     'General practicioner':'act_GP_region_brand_1mo', 'Internal medicine':'act_IM_region_brand_1mo',
                                                     'Internal medicine / pneumology':'act_IMP_region_brand_1mo',
                                                     'Internal medicine and general practicioner':'act_IMPG_region_brand_1mo',
                                                     'Pediatrician':'act_PE_region_brand_1mo'})
    
    activity_features.month = activity_features.month.apply(lambda x: add_month(x))
    # Add rolling aggregations of the previously created features
    
    cols = [c for c in activity_features.columns if c not in ['month', 'region', 'brand']]
    
    for c in cols:
        activity_features[c[:-3] + '2mo'] = activity_features.groupby(['region', 'brand'])[c].shift()
        activity_features[c[:-3] + '3mo'] = activity_features.groupby(['region', 'brand'])[c].shift(2)
        activity_features[c[:-3] + '4mo'] = activity_features.groupby(['region', 'brand'])[c].shift(3)
        
        activity_features[ c[:-3] + 'trend_2mo'] = activity_features[c[:-3] + '2mo'] / (activity_features[c[:-3] + '1mo']+1)
        activity_features[c[:-3] + 'trend_3mo'] = activity_features[c[:-3] + '3mo'] / (activity_features[c[:-3] + '1mo']+1)
        activity_features[ c[:-3] + 'trend_4mo'] = activity_features[c[:-3] + '4mo'] / (activity_features[c[:-3] + '1mo']+1)
    
    return activity_features

In [42]:
#checking 
raw_activity[(raw_activity.month=='2020-10')&(raw_activity.region=='region_50')&
             (raw_activity.channel=='f2f')&(raw_activity.brand=='brand_2')].counts.sum()

6

In [39]:
#checking
temp = extract_activity_features(raw_activity)
temp[(temp.brand=='brand_2')&(temp.region=='region_50')][['month', 'brand', 'region', 
                                                           'act_f2f_region_brand_1mo','act_f2f_region_brand_2mo', 'act_f2f_region_brand_3mo']]

,month,brand,region,act_f2f_region_brand_1mo,act_f2f_region_brand_2mo,act_f2f_region_brand_3mo
152,2020-08,brand_2,region_50,8.0,NaN,NaN
535,2020-09,brand_2,region_50,4.0,8.0,NaN
926,2020-10,brand_2,region_50,10.0,4.0,8.0
1316,2020-11,brand_2,region_50,6.0,10.0,4.0
2855,2021-05,brand_2,region_50,0.0,6.0,10.0
3119,2021-06,brand_2,region_50,3.0,0.0,6.0
3443,2021-07,brand_2,region_50,7.0,3.0,0.0
3783,2021-08,brand_2,region_50,13.0,7.0,3.0
4093,2021-09,brand_2,region_50,3.0,13.0,7.0
4398,2021-10,brand_2,region_50,2.0,3.0,13.0
